In [1]:
from sklearn.preprocessing import StandardScaler

from influxdb.commands import InfluxDBQueries, DataProcessor
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='10m', pair='BTCUSDT', start="2022-01-01T01:00:00Z",stop="2025-01-01T01:00:00Z")

In [3]:
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='10m', pair='BTCUSDT', start="2025-01-01T01:00:00Z", stop="2025-04-01T01:00:00Z")

In [4]:
df_combined = pd.concat([df_train, df_test])
original_idx = df_train.shape[0]

In [5]:
df_combined, scaler, columns = DataProcessor.add_features(df_combined)

In [6]:
df_train = df_combined.iloc[:original_idx]
df_test = df_combined.iloc[original_idx:]

In [7]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [8]:
X_train, y_train = DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=288, forecast_horizon=12)

In [9]:
from influxdb.commands import LSTMModel

In [10]:
X_train.shape

(157458, 288, 29)

Based on the previous cells in the notebook, I can see that `X_train.shape` would reveal the correct shape for the input. Looking at how the data is processed, I need to match the shape with the columns used in the LSTM model.



In [11]:
lstm_forecaster = LSTMModel(input_shape=(288, len(columns)), output_steps=12, lstm_units=30)

In [ ]:
lstm_forecaster.fit(X_train, y_train, epochs=30, batch_size=64, patience=6)

Training started with EarlyStopping (patience=6)...


In [13]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=12)

In [14]:
X_test.shape

(1837, 288, 29)

In [15]:
pred = lstm_forecaster.predict(X_test)

58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step


In [20]:
pred[0]

array([[0.41146407],
       [0.43979862],
       [0.35038003],
       [0.37909704],
       [0.36247152],
       [0.35263857],
       [0.4109227 ],
       [0.36012414],
       [0.32200548],
       [0.30043164],
       [0.28236005],
       [0.3760441 ]], dtype=float32)

In [21]:
df_test_pred = df_test

In [26]:
df_test_pred = df_test.iloc[:12]

In [28]:
df_test_pred['close'] = pred[0]

In [35]:
df_test_pred[columns] = scaler.inverse_transform(df_test_pred[columns])

In [36]:
df_test_pred

,high,low,close,volume,open,close_lag_1,close_lag_2,close_lag_3,close_lag_4,close_lag_5,...,ma_30,rsi_14,bb_mean_20,bb_std_20,bb_hband,bb_lband,macd,macd_signal,atr,obv
time,,,,,,,,,,,,,,,,,,,,,
2025-01-01 02:00:00+00:00,94408.72,93578.77,54956.588430,534.68092,94401.13,94401.14,93576.00,93488.84,93899.68,93564.04,...,93655.835667,45.262879,94138.3940,787.419417,95713.232833,92563.555167,74.226613,169.524609,875.155714,476373.902016
2025-01-01 03:00:00+00:00,94105.12,93594.56,55646.336729,245.77118,93607.74,93607.74,94401.14,93576.00,93488.84,93899.68,...,93647.839333,46.748326,94202.5395,723.700200,95649.939900,92755.139100,79.206969,151.461081,859.754286,476619.673196
2025-01-01 04:00:00+00:00,94098.91,93728.22,53469.619505,199.32214,94098.90,94098.91,93607.74,94401.14,93576.00,93488.84,...,93706.574000,35.409256,94251.4040,658.379336,95568.162671,92934.645329,61.396172,133.448099,776.920714,476420.351056
2025-01-01 05:00:00+00:00,93838.04,93500.00,54168.677865,251.77882,93838.04,93838.04,94098.91,93607.74,94401.14,93576.00,...,93732.212667,32.821684,94232.4305,673.324915,95579.080330,92885.780670,24.076531,111.573786,738.377143,476168.572236
2025-01-01 06:00:00+00:00,93937.19,93547.40,53763.962655,184.73424,93553.91,93553.91,93838.04,94098.91,93607.74,94401.14,...,93765.545000,36.844020,94220.8375,679.076096,95578.989691,92862.685309,13.557650,91.970558,689.540714,476353.306476
2025-01-01 07:00:00+00:00,93792.02,93568.38,53524.599137,245.68231,93792.02,93792.02,93553.91,93838.04,94098.91,93607.74,...,93809.131000,40.165519,94203.0320,686.659677,95576.351355,92829.712645,2.414507,74.059348,635.220714,476107.624166
2025-01-01 08:00:00+00:00,93800.00,93664.08,54943.410120,178.10766,93757.57,93757.58,93792.02,93553.91,93838.04,94098.91,...,93850.748667,45.608483,94175.6030,696.302005,95568.207009,92782.998991,-12.205043,56.806470,512.206429,475929.516506
2025-01-01 09:00:00+00:00,93756.02,93304.00,53706.820372,429.45973,93684.11,93684.10,93757.58,93792.02,93553.91,93838.04,...,93882.485667,45.513497,94120.0335,709.917732,95539.868965,92700.198035,-43.912953,36.662585,498.999286,475500.056776
2025-01-01 10:00:00+00:00,93486.29,92888.00,52778.897614,660.25220,93428.47,93428.46,93684.10,93757.58,93792.02,93553.91,...,93912.601333,41.994707,94011.3305,636.057861,95283.446221,92739.214779,-69.483389,15.433390,500.415714,474839.804576


In [38]:
df_test[columns] = scaler.inverse_transform(df_test[columns])

In [39]:
df_test

,high,low,close,volume,open,close_lag_1,close_lag_2,close_lag_3,close_lag_4,close_lag_5,...,ma_30,rsi_14,bb_mean_20,bb_std_20,bb_hband,bb_lband,macd,macd_signal,atr,obv
time,,,,,,,,,,,,,,,,,,,,,
2025-01-01 02:00:00+00:00,94408.72,93578.77,93607.74,534.68092,94401.13,94401.14,93576.00,93488.84,93899.68,93564.04,...,93655.835667,45.262879,94138.3940,787.419417,95713.232833,92563.555167,74.226613,169.524609,875.155714,476373.902016
2025-01-01 03:00:00+00:00,94105.12,93594.56,94098.91,245.77118,93607.74,93607.74,94401.14,93576.00,93488.84,93899.68,...,93647.839333,46.748326,94202.5395,723.700200,95649.939900,92755.139100,79.206969,151.461081,859.754286,476619.673196
2025-01-01 04:00:00+00:00,94098.91,93728.22,93838.04,199.32214,94098.90,94098.91,93607.74,94401.14,93576.00,93488.84,...,93706.574000,35.409256,94251.4040,658.379336,95568.162671,92934.645329,61.396172,133.448099,776.920714,476420.351056
2025-01-01 05:00:00+00:00,93838.04,93500.00,93553.91,251.77882,93838.04,93838.04,94098.91,93607.74,94401.14,93576.00,...,93732.212667,32.821684,94232.4305,673.324915,95579.080330,92885.780670,24.076531,111.573786,738.377143,476168.572236
2025-01-01 06:00:00+00:00,93937.19,93547.40,93792.02,184.73424,93553.91,93553.91,93838.04,94098.91,93607.74,94401.14,...,93765.545000,36.844020,94220.8375,679.076096,95578.989691,92862.685309,13.557650,91.970558,689.540714,476353.306476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31 21:00:00+00:00,82846.20,82350.84,82431.32,405.06808,82462.26,82462.27,83291.42,82680.34,83288.89,83418.78,...,82382.095333,53.470876,82374.4000,614.074621,83602.549241,81146.250759,93.873003,41.382430,787.646429,443463.445446
2025-03-31 22:00:00+00:00,82752.18,82422.00,82563.99,287.29447,82431.32,82431.32,82462.27,83291.42,82680.34,83288.89,...,82376.576000,55.767584,82404.9480,607.167420,83619.282840,81190.613160,78.028046,48.711553,770.313571,443750.739916
2025-03-31 23:00:00+00:00,82571.98,82265.10,82409.99,209.71441,82563.99,82563.99,82431.32,82462.27,83291.42,82680.34,...,82380.775333,54.573027,82429.3325,596.462267,83622.257035,81236.407965,52.439797,49.457202,764.470000,443541.025506
